In [1]:
import requests
import csv
from datetime import datetime

keys, transactions = None, []
current_time = 1654016400
last_transactions = set()

while True:
    query = """
    {
        borrows(where: {timestamp_lte: """ + str(current_time) + """}, first: 1000, orderBy: timestamp, orderDirection: desc) {
            id
            user {
                id
            }
            caller {
                id
            }
            reserve {
                symbol
            }
            amount
            borrowRate
            borrowRateMode
            timestamp
            stableTokenDebt
            variableTokenDebt
        }
    }
    """
    response = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                                '',
                                json={'query': query})

    if response.status_code != 200:
        print("Problem reading from timestamp", current_time, ":", response.status_code)
        continue
    
    try:
        data = response.json()["data"]["borrows"]
    except Exception:
        print("Error at timestamp", current_time)
        continue

    if len(data) == 0:
        break
    
    if keys is None:
        keys = data[0].keys()

    index = 0
    try:
        while data[index]["id"] in last_transactions:
            index += 1
    except IndexError:
            current_time -= 1
            continue
    print(len(data) - index, "transactions found at timestamp", current_time)

    for transaction in data[index:]:
        transaction["user"] = transaction["user"]["id"]
        transaction["reserve"] = transaction["reserve"]["symbol"]
        transaction["caller"] = transaction["caller"]["id"]
        transactions.append(transaction)

    current_time = int(data[-1]["timestamp"])
    index = -1
    last_transactions = set()
    while data[index]["timestamp"] == data[-1]["timestamp"]:
        last_transactions.add(data[index]["id"])
        index -= 1

with open('borrow.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(transactions)
        

1000 transactions found at timestamp 1654016400
999 transactions found at timestamp 1653557113
999 transactions found at timestamp 1653068815
999 transactions found at timestamp 1652706002
999 transactions found at timestamp 1652476109
999 transactions found at timestamp 1652218168
999 transactions found at timestamp 1652037410
998 transactions found at timestamp 1652000128
999 transactions found at timestamp 1651964114
998 transactions found at timestamp 1651918172
999 transactions found at timestamp 1651892623
999 transactions found at timestamp 1651740844
999 transactions found at timestamp 1651149587
999 transactions found at timestamp 1650594069
999 transactions found at timestamp 1650117272
999 transactions found at timestamp 1649613509
999 transactions found at timestamp 1649239147
999 transactions found at timestamp 1648785756
999 transactions found at timestamp 1648423162
999 transactions found at timestamp 1648168458
999 transactions found at timestamp 1647646020
999 transact

In [2]:
!wc -l borrow.csv
# 152726

154961 borrow.csv


In [3]:
from google.colab import files
files.download('borrow.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>